<a href="https://colab.research.google.com/github/juliombb/mc886-linear-regression/blob/master/Trab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MC 886 - Regressão linear - Trabalho 1


> Júlio M. B. Barros RA: 200491

> Paulo Martins RA: xxxxxx



In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [89]:
df = pd.read_csv("Metro_Interstate_Traffic_Volume.csv")
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


In [0]:
%%capture
df.pop('weather_description')

In [91]:
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,2012-10-02 09:00:00,5545
1,None,289.36,0.0,0.0,75,Clouds,2012-10-02 10:00:00,4516
2,None,289.58,0.0,0.0,90,Clouds,2012-10-02 11:00:00,4767
3,None,290.13,0.0,0.0,90,Clouds,2012-10-02 12:00:00,5026
4,None,291.14,0.0,0.0,75,Clouds,2012-10-02 13:00:00,4918


In [92]:
df.loc[df['holiday'] != "None", 'holiday'] = 1 # acesso em localização via boolean array
df.loc[df['holiday'] == "None", 'holiday'] = 0
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,date_time,traffic_volume
0,0,288.28,0.0,0.0,40,Clouds,2012-10-02 09:00:00,5545
1,0,289.36,0.0,0.0,75,Clouds,2012-10-02 10:00:00,4516
2,0,289.58,0.0,0.0,90,Clouds,2012-10-02 11:00:00,4767
3,0,290.13,0.0,0.0,90,Clouds,2012-10-02 12:00:00,5026
4,0,291.14,0.0,0.0,75,Clouds,2012-10-02 13:00:00,4918


In [93]:
## Todas as categorias do dataset inclusas, decidir se precisa categorizar depois
df_weather = pd.get_dummies(df['weather_main']) ## Nao sei se pode usar isso < provavelmente pode
df_weather.head()

,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm
0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0


In [94]:
df = df.join(df_weather)
df.pop('weather_main')
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,date_time,traffic_volume,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm
0,0,288.28,0.0,0.0,40,2012-10-02 09:00:00,5545,0,1,0,0,0,0,0,0,0,0,0
1,0,289.36,0.0,0.0,75,2012-10-02 10:00:00,4516,0,1,0,0,0,0,0,0,0,0,0
2,0,289.58,0.0,0.0,90,2012-10-02 11:00:00,4767,0,1,0,0,0,0,0,0,0,0,0
3,0,290.13,0.0,0.0,90,2012-10-02 12:00:00,5026,0,1,0,0,0,0,0,0,0,0,0
4,0,291.14,0.0,0.0,75,2012-10-02 13:00:00,4918,0,1,0,0,0,0,0,0,0,0,0


In [0]:
df_date = pd.to_datetime(df['date_time']).dt.date
df_time = pd.to_datetime(df['date_time']).dt.time

In [96]:
df_date.head()

0    2012-10-02
1    2012-10-02
2    2012-10-02
3    2012-10-02
4    2012-10-02
Name: date_time, dtype: object

In [0]:
import datetime
df_date = pd.DataFrame({'weekday': [d.weekday() for d in df_date]}) ## d.weekday() -> segunda eh 0 e domingo, 6

In [98]:
df_date.loc[df_date['weekday'] < 5] = 0
df_date.loc[df_date['weekday'] >= 5] = 1
df_date['weekday'].unique()

array([0, 1])

In [99]:
df = df.join(pd.DataFrame({'weekend': df_date['weekday']}))
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,date_time,traffic_volume,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm,weekend
0,0,288.28,0.0,0.0,40,2012-10-02 09:00:00,5545,0,1,0,0,0,0,0,0,0,0,0,0
1,0,289.36,0.0,0.0,75,2012-10-02 10:00:00,4516,0,1,0,0,0,0,0,0,0,0,0,0
2,0,289.58,0.0,0.0,90,2012-10-02 11:00:00,4767,0,1,0,0,0,0,0,0,0,0,0,0
3,0,290.13,0.0,0.0,90,2012-10-02 12:00:00,5026,0,1,0,0,0,0,0,0,0,0,0,0
4,0,291.14,0.0,0.0,75,2012-10-02 13:00:00,4918,0,1,0,0,0,0,0,0,0,0,0,0


In [100]:
df = df.join(pd.DataFrame({'hour_of_day': df_time.map(lambda t : t.hour)}))
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,date_time,traffic_volume,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm,weekend,hour_of_day
0,0,288.28,0.0,0.0,40,2012-10-02 09:00:00,5545,0,1,0,0,0,0,0,0,0,0,0,0,9
1,0,289.36,0.0,0.0,75,2012-10-02 10:00:00,4516,0,1,0,0,0,0,0,0,0,0,0,0,10
2,0,289.58,0.0,0.0,90,2012-10-02 11:00:00,4767,0,1,0,0,0,0,0,0,0,0,0,0,11
3,0,290.13,0.0,0.0,90,2012-10-02 12:00:00,5026,0,1,0,0,0,0,0,0,0,0,0,0,12
4,0,291.14,0.0,0.0,75,2012-10-02 13:00:00,4918,0,1,0,0,0,0,0,0,0,0,0,0,13


In [101]:
df.pop('date_time')
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm,weekend,hour_of_day
0,0,288.28,0.0,0.0,40,5545,0,1,0,0,0,0,0,0,0,0,0,0,9
1,0,289.36,0.0,0.0,75,4516,0,1,0,0,0,0,0,0,0,0,0,0,10
2,0,289.58,0.0,0.0,90,4767,0,1,0,0,0,0,0,0,0,0,0,0,11
3,0,290.13,0.0,0.0,90,5026,0,1,0,0,0,0,0,0,0,0,0,0,12
4,0,291.14,0.0,0.0,75,4918,0,1,0,0,0,0,0,0,0,0,0,0,13


Now that we have only columns we want and with hot encoding, let's normalize everything to keep the mean near zero

In [102]:
t_mean = df['temp'].mean()
t_std = df['temp'].std()
df['temp'] = df['temp'].map(lambda temp: (temp-t_mean)/t_std)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm,weekend,hour_of_day
0,0,0.530365,0.0,0.0,40,5545,0,1,0,0,0,0,0,0,0,0,0,0,9
1,0,0.611335,0.0,0.0,75,4516,0,1,0,0,0,0,0,0,0,0,0,0,10
2,0,0.627829,0.0,0.0,90,4767,0,1,0,0,0,0,0,0,0,0,0,0,11
3,0,0.669064,0.0,0.0,90,5026,0,1,0,0,0,0,0,0,0,0,0,0,12
4,0,0.744786,0.0,0.0,75,4918,0,1,0,0,0,0,0,0,0,0,0,0,13


In [103]:
c_mean = df['clouds_all'].mean()
c_std = df['clouds_all'].std()
df['clouds_all'] = df['clouds_all'].map(lambda clouds_all: (clouds_all-c_mean)/c_std)

v_mean = df['traffic_volume'].mean()
v_std = df['traffic_volume'].std()
df['traffic_volume'] = df['traffic_volume'].map(lambda traffic_volume: (traffic_volume-v_mean)/v_std)

h_mean = df['hour_of_day'].mean()
h_std = df['hour_of_day'].std()
df['hour_of_day'] = df['hour_of_day'].map(lambda hour_of_day: (hour_of_day-h_mean)/h_std)

df.head()


,holiday,temp,rain_1h,snow_1h,clouds_all,traffic_volume,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm,weekend,hour_of_day
0,0,0.530365,0.0,0.0,-0.239960,1.150147,0,1,0,0,0,0,0,0,0,0,0,0,-0.345545
1,0,0.611335,0.0,0.0,0.657113,0.632244,0,1,0,0,0,0,0,0,0,0,0,0,-0.201457
2,0,0.627829,0.0,0.0,1.041573,0.758574,0,1,0,0,0,0,0,0,0,0,0,0,-0.057370
3,0,0.669064,0.0,0.0,1.041573,0.888931,0,1,0,0,0,0,0,0,0,0,0,0,0.086717
4,0,0.744786,0.0,0.0,0.657113,0.834574,0,1,0,0,0,0,0,0,0,0,0,0,0.230804


Separating our data in train, validation and test...

In [0]:
train = df.iloc[:int(.6*len(df))] ## primeiros 60% do dataframe
validation = df.iloc[int(.6*len(df)):int(.8*len(df))] ## proximos 20% do dataframe
test = df.iloc[int(.8*len(df)):] ## ultimos 20% do dataframe

In [0]:
target_train = train.pop('traffic_volume')
target_validation = validation.pop('traffic_volume')
target_test = test.pop('traffic_volume')

In [106]:
train.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,Clear,Clouds,Drizzle,Fog,Haze,Mist,Rain,Smoke,Snow,Squall,Thunderstorm,weekend,hour_of_day
0,0,0.530365,0.0,0.0,-0.239960,0,1,0,0,0,0,0,0,0,0,0,0,-0.345545
1,0,0.611335,0.0,0.0,0.657113,0,1,0,0,0,0,0,0,0,0,0,0,-0.201457
2,0,0.627829,0.0,0.0,1.041573,0,1,0,0,0,0,0,0,0,0,0,0,-0.057370
3,0,0.669064,0.0,0.0,1.041573,0,1,0,0,0,0,0,0,0,0,0,0,0.086717
4,0,0.744786,0.0,0.0,0.657113,0,1,0,0,0,0,0,0,0,0,0,0,0.230804


In [107]:
target_train.head()

0    1.150147
1    0.632244
2    0.758574
3    0.888931
4    0.834574
Name: traffic_volume, dtype: float64

Testing our model against sklearn `SGDRegressor`


In [108]:
from sklearn import linear_model
regressor = linear_model.SGDRegressor(max_iter=500000)
regressor.fit(train.to_numpy(), target_train.to_numpy())

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=500000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [109]:
prediction = regressor.predict(validation.to_numpy())
prediction

array([-2.22040848e+07, -1.82547501e+07, -1.78071399e+07, ...,
        6.72878947e+07,  2.39300518e+08,  2.03740025e+07])

In [110]:
target_validation.to_numpy()

array([ 1.43803825,  1.34593315,  1.01475744, ..., -0.10811949,
       -0.10811949, -0.27974702])

In [113]:
np.subtract(prediction, target_validation.to_numpy())

array([-2.22040863e+07, -1.82547514e+07, -1.78071409e+07, ...,
        6.72878949e+07,  2.39300518e+08,  2.03740027e+07])

Deu ruim :c
